In [69]:
import pandas as pd
import json
import sqlite3
import re
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [71]:
# Step 1: Load CSV Data

orders_path = 'orders.csv'
orders_df = pd.read_csv(orders_path)
orders_df.columns = [column.strip() for column in orders_df.columns]

print(orders_df.shape)
orders_df.head()

(10000, 6)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [72]:
# Step 2: Load JSON Data

users_path = 'users.json'

try:
    users_df = pd.read_json(users_path, lines=True)
except ValueError:
    with open(users_path, 'r', encoding='utf-8') as f:
        parsed_json = json.load(f)
    users_df = pd.json_normalize(parsed_json)

users_df.columns = [column.strip() for column in users_df.columns]

print(users_df.shape)
users_df.head()

(3000, 4)


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [70]:
# Step 3: Load SQL file

with open(path, 'r', encoding='utf-8') as f:
  sql = f.read()

conn = sqlite3.connect(":memory:")
conn.executescript(Path(path).read_text(encoding="utf-8"))

restaurants_df = pd.read_sql_query("SELECT * FROM restaurants;", conn)
conn.close()

restaurants_df.columns = [c.strip() for c in restaurants_df.columns]

print(restaurants_df.shape)
restaurants_df.head()

(500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [73]:
# Step 4: Merge the Data

def find_column(df, candidates):
    for col in df.columns:
        if col.lower() in [c.lower() for c in candidates]:
            return col
    return None

# Join Keys
order_user_key = find_column(orders_df, ['user_id', 'userid', 'userId'])
user_key = find_column(users_df, ['user_id', 'userid', 'userId'])
order_rest_key = find_column(orders_df, ['restaurant_id', 'restaurantid', 'restaurantId'])
rest_key = find_column(restaurants_df, ['restaurant_id', 'restaurantid', 'restaurantId'])

merged_df = (
    orders_df
    .merge(users_df, how='left', left_on=order_user_key, right_on=user_key, suffixes=('', '_user'))
    .merge(restaurants_df, how='left', left_on=order_rest_key, right_on=rest_key, suffixes=('', '_rest'))
)

print("Merged Dataset:", merged_df.shape)
merged_df.head()

Merged Dataset: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_rest,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [74]:
# Step 5: Create Final Dataset

merged_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("final_food_delivery_dataset.csv")

def detect_datetime_col(df):
    for column in df.columns:
        if any(keyword in column.lower() for keyword in ['date', 'time', 'created', 'ordered']):
            try:
                pd.to_datetime(df[column])
                return column
            except:
                continue
    return None

def detect_amount_col(df):
    preferred_keywords = ['order_value', 'amount', 'total', 'price']
    for keyword in preferred_keywords:
        for column in df.columns:
            if keyword in column.lower():
                return column
    return df.select_dtypes(include=[np.number]).columns[0]

def detect_membership_col(df):
    return next((column for column in df.columns if 'member' in column.lower() or 'gold' in column.lower()), None)

def detect_city_col(df):
    return next((column for column in df.columns if 'city' in column.lower()), None)

def detect_cuisine_col(df):
    return next((column for column in df.columns if 'cuisine' in column.lower()), None)

def plot_order_trend(df):
    datetime_col = detect_datetime_col(df)
    df[datetime_col] = pd.to_datetime(df[datetime_col], dayfirst=True)
    daily_order_counts = df.groupby(df[datetime_col].dt.date).size()
    daily_order_counts.plot(title='Order Trend Over Time', xlabel='Date', ylabel='Order Count', rot=45)
    plt.tight_layout()
    plt.show()
    return daily_order_counts

final_food_delivery_dataset.csv


In [75]:
# Identify required columns
membership_column = detect_membership_col(merged_df)
city_column = detect_city_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Filter orders placed by Gold members
gold_member_orders = merged_df[merged_df[membership_column].str.strip().str.lower() == 'gold']

# Aggregate total revenue by city
gold_revenue_by_city = (
    gold_member_orders
    .groupby(city_column)[order_amount_column]
    .sum()
    .sort_values(ascending=False)
)

# Filter Cities
target_cities = ['Hyderabad', 'Bangalore', 'Chennai', 'Pune']
filtered_revenue = gold_revenue_by_city.loc[gold_revenue_by_city.index.isin(target_cities)]

print("Gold Members Revenue (Selected Cities):")
print(filtered_revenue.to_string())

Gold Members Revenue (Selected Cities):
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19


In [76]:
cuisine_column = detect_cuisine_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Compute Average Order Value by cuisine
average_order_value_by_cuisine = (
    merged_df
    .groupby(cuisine_column)[order_amount_column]
    .mean()
    .sort_values(ascending=False)
)

# Filter Cuisines
target_cuisines = ['Indian', 'Chinese', 'Italian', 'Mexican']
filtered_avg_order_value = average_order_value_by_cuisine.loc[average_order_value_by_cuisine.index.isin(target_cuisines)]

print("Average Order Value by Cuisine:")
print(filtered_avg_order_value.to_string())

Average Order Value by Cuisine:
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020


In [77]:
user_id_column = find_column(merged_df, ['user_id', 'userid'])
order_amount_column = detect_amount_col(merged_df)

# Aggregate Total Order Value per User
total_order_value_by_user = (merged_df.groupby(user_id_column)[order_amount_column].sum())

# Filter users with cumulative order value bigger than 1000
high_value_user_count = (total_order_value_by_user > 1000).sum()

print("Users with total orders > ₹1000:", high_value_user_count)

Users with total orders > ₹1000: 2544


In [78]:
restaurant_rating_column = next((col for col in merged_df.columns if 'rating' in col.lower()), None)
order_amount_column = detect_amount_col(merged_df)

# Filter out Rows with Missing Ratings
valid_rating_data = merged_df[merged_df[restaurant_rating_column].notna()].copy()

# Define Rating Intervals
rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

# Categorize Rating into Defined Bins
valid_rating_data['rating_range'] = pd.cut(
    valid_rating_data[restaurant_rating_column],
    bins=rating_bins,
    labels=rating_labels,
    include_lowest=True
)

# Aggregate Total Revenue by Rating Range
total_revenue_by_rating = (
    valid_rating_data
    .groupby('rating_range', observed=True)[order_amount_column]
    .sum()
    .sort_values(ascending=False)
)

print("Total Revenue by Restaurant Rating Range:")
print(total_revenue_by_rating.to_string())

Total Revenue by Restaurant Rating Range:
rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41


In [47]:
membership_column = detect_membership_col(merged_df)
city_column = detect_city_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Filter Orders placed by Gold members
gold_orders = merged_df[
    merged_df[membership_column].str.strip().str.lower() == 'gold'
]

# Calculate Average Order Value by City
avg_order_value_by_city = (
    gold_orders
    .groupby(city_column, observed=True)[order_amount_column]
    .mean()
    .sort_values(ascending=False)
)

# Compare Target Cities
target_cities = ['Hyderabad', 'Bangalore', 'Chennai', 'Pune']
filtered_avg = avg_order_value_by_city.loc[avg_order_value_by_city.index.isin(target_cities)]

print("Average Order Value by Gold Members City:")
print(filtered_avg.to_string())

Average Order Value by Gold Members City:
city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243


In [49]:
cuisine_column = detect_cuisine_col(merged_df)
restaurant_id_column = find_column(merged_df, ['restaurant_id', 'restaurantid'])
order_amount_column = detect_amount_col(merged_df)

# Aggregate Number of Distinct Restaurants and Total Revenue by cuisine
cuisine_summary = (
    merged_df
    .groupby(cuisine_column, observed=True)
    .agg(
        number_of_restaurants=(restaurant_id_column, 'nunique'),
        total_revenue=(order_amount_column, 'sum')
    )
    .sort_values(by='number_of_restaurants')
)

# Filter Target Cuisines
target_cuisines = ['Indian', 'Chinese', 'Italian', 'Mexican']
filtered_cuisine_stats = cuisine_summary.loc[cuisine_summary.index.isin(target_cuisines)]

print("Cuisine Statistics:")
print(filtered_cuisine_stats.to_string())

Cuisine Statistics:
         number_of_restaurants  total_revenue
cuisine                                      
Chinese                    120     1930504.65
Indian                     126     1971412.58
Italian                    126     2024203.80
Mexican                    128     2085503.09


In [62]:
membership_column = detect_membership_col(merged_df)

# Total Number of Orders
total_orders = merged_df.shape[0]

# Orders placed by Gold Members
gold_orders = merged_df[merged_df[membership_column].str.lower() == 'gold'].shape[0]
gold_order_percentage = round((gold_orders / total_orders) * 100)

print(f"Gold Membership Order Ratio: {gold_order_percentage}%")

Gold Membership Order Ratio: 50%


In [64]:
restaurant_name_column = next((col for col in merged_df.columns if 'name' in col.lower()), None)
restaurant_id_column = find_column(merged_df, ['restaurant_id', 'restaurantid'])
order_amount_column = detect_amount_col(merged_df)

# Aggregate Order Count with Average Order Value by Restaurant
restaurant_metrics = (
    merged_df
    .groupby(restaurant_name_column, observed=True)
    .agg(
        order_count=(restaurant_id_column, 'count'),
        average_order_value=(order_amount_column, 'mean')
    )
)

# Filter Restaurants Orders with Less than 20
low_volume_restaurants = restaurant_metrics[restaurant_metrics['order_count'] < 20]

# Define Target Restaurants
target_restaurants = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

result = (low_volume_restaurants.loc[low_volume_restaurants.index.isin(target_restaurants)].sort_values(by='average_order_value', ascending=False))

print("Target Restaurants:")
print(result.to_string())

Target Restaurants:
                     order_count  average_order_value
restaurant_name                                      
Ruchi Foods Chinese           19           686.603158


In [63]:
membership_column = detect_membership_col(merged_df)
cuisine_column = detect_cuisine_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Normalize Text
merged_df['membership_clean'] = merged_df[membership_column].str.strip().str.lower()
merged_df['cuisine_clean'] = merged_df[cuisine_column].str.strip().str.lower()

# Create Combination Field
merged_df['membership_cuisine_combo'] = merged_df['membership_clean'] + ' + ' + merged_df['cuisine_clean']

# Aggregate Revenue
revenue_by_combo = (
    merged_df
    .groupby('membership_cuisine_combo', observed=True)[order_amount_column]
    .sum()
    .sort_values(ascending=False)
)

# Filter Target Combinations
target_combos = [
    'gold + indian',
    'gold + italian',
    'regular + indian',
    'regular + chinese'
]

for combo in target_combos:
    value = revenue_by_combo.get(combo)
    if pd.notna(value):
        print(f"{combo}\t₹{value:,.2f}")

gold + indian	₹979,312.31
gold + italian	₹1,005,779.05
regular + indian	₹992,100.27
regular + chinese	₹952,790.91


In [58]:
date_column = detect_datetime_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Parse Date Column
merged_df[date_column] = pd.to_datetime(merged_df[date_column], format='%d-%m-%Y', errors='coerce')

# Create Quarterly Tag
merged_df['order_quarter'] = merged_df[date_column].dt.to_period('Q').astype(str)

# Sum Revenue by Quarter
revenue_by_quarter = (
    merged_df
    .groupby('order_quarter', observed=True)[order_amount_column]
    .sum()
    .sort_values(ascending=False)
)

for quarter, revenue in revenue_by_quarter.items():
    print(f"{quarter}\t₹{revenue:,.2f}")

2023Q3	₹2,037,385.10
2023Q4	₹2,018,263.66
2023Q1	₹1,993,425.14
2023Q2	₹1,945,348.72
2024Q1	₹17,201.50


In [79]:
membership_column = detect_membership_col(merged_df)

# Count Total Rows
gold_order_count = merged_df[merged_df[membership_column].str.strip().str.lower() == 'gold'].shape[0]

print(f"Total Orders by Gold Members: {gold_order_count}")

Total Orders by Gold Members: 4987


In [55]:
city_column = detect_city_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Filter for Orders placed in Hyderabad
hyderabad_orders = merged_df[merged_df[city_column].str.strip().str.lower() == 'hyderabad']

# Sum the Total Revenue
total_revenue_hyderabad = round(hyderabad_orders[order_amount_column].sum())

print(f"Total Revenue from Hyderabad (Rounded to Nearest Integer): ₹{total_revenue_hyderabad}")

Total Revenue from Hyderabad (Rounded to Nearest Integer): ₹1889367


In [29]:
# Identify the user ID column
user_id_column = find_column(merged_df, ['user_id', 'userid', 'userId'])

# Count unique users with at least one order
number_of_unique_users = merged_df[user_id_column].nunique()

print(f"Distinct users who placed at least one order: {number_of_unique_users}")

Distinct users who placed at least one order: 2883


In [52]:
membership_column = detect_membership_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Filter Gold Member Orders
gold_orders = merged_df[merged_df[membership_column].str.strip().str.lower() == 'gold']

# Calculate Average Order Value for Gold Members
average_order_value = round(gold_orders[order_amount_column].mean(), 2)

print(f"Average Order Value for Gold Members: ₹{average_order_value}")

Average Order Value for Gold Members: ₹797.15


In [51]:
restaurant_rating_column = next((col for col in merged_df.columns if 'rating' in col.lower()), None)

# Filter orders for Restaurants with Rating bigger than & equal to 4.5
high_rating_orders = merged_df[merged_df[restaurant_rating_column] >= 4.5]

# Count Number of Qualifying orders
order_count = high_rating_orders.shape[0]

print(f"Number of orders for restaurants rated ≥ 4.5: {order_count}")

Number of orders for restaurants rated ≥ 4.5: 3374


In [53]:
membership_column = detect_membership_col(merged_df)
city_column = detect_city_col(merged_df)
order_amount_column = detect_amount_col(merged_df)

# Filter Gold Member orders
gold_orders = merged_df[merged_df[membership_column].str.strip().str.lower() == 'gold']

# Aggregate Revenue by city
revenue_by_city = (
    gold_orders
    .groupby(city_column, observed=True)[order_amount_column]
    .sum()
    .sort_values(ascending=False)
)

top_revenue_city = revenue_by_city.idxmax()
gold_orders_in_top_city = gold_orders[gold_orders[city_column] == top_revenue_city].shape[0]

print(f"Number of orders in {top_revenue_city}: {gold_orders_in_top_city}")

Number of orders in Chennai: 1337
